In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import json
from time import sleep

In [2]:
newspaper_name = 'GazetaBT'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

url_pages = 'https://www.gazetabt.ro/cat/local'

# categs = ['cultural', 'ecologic', 'economic', 'interviu', 'social']

num_arts = 300


In [3]:
def get_all_articles(url: str, headers: dict) -> list:
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    # print(soup.prettify())
    # Find <h2 class='post-title'><a href='URL'>TITLE</a></h2>
    main_content = soup.find('div', class_='container-wrapper')
    # print(main_content)
    articles = main_content.find_all('h2', class_='post-title')
    # print(articles[0])
    
    # result = []
    # for article in articles:
    #     category = article.find('span', class_='article-categories').find('a').get('href')
    #     # print(category)
    #     if category == allowed_categ:
    #         result.append({'url': article.find('a').get('href'), 'title': article.find('a').get('title')})
    
    articles = [{'url': article.find('a').get('href'), 'title': article.find('a').text} for article in articles]
    
    return articles

In [4]:
get_all_articles(url_pages + '/page/2', headers)

[{'url': 'https://www.gazetabt.ro/un-nou-cod-galben-de-ceata-si-polei-in-judet-botosani/',
  'title': 'Un nou Cod galben de ceață și polei în județ Botoșani'},
 {'url': 'https://www.gazetabt.ro/muzeul-judetean-botosani-participant-la-vernisajul-expozitiei-tehnici-traditionale-de-tesere-a-scoartelor-in-romania-si-republica-moldova/',
  'title': 'Muzeul Județean Botoșani, participant la vernisajul expoziției „Tehnici tradiționale de țesere a scoarțelor în România și Republica Moldova”'},
 {'url': 'https://www.gazetabt.ro/elicopterul-smurd-chemat-la-botosani-pentru-a-prelua-un-barbat-in-stare-grava-14/',
  'title': 'Elicopterul SMURD chemat la Botoșani pentru a prelua un bărbat în stare gravă'},
 {'url': 'https://www.gazetabt.ro/nova-apaserv-anunta-sistarea-apei-potabile-in-municipiul-botosani/',
  'title': 'Nova Apaserv anunță sistarea apei potabile în municipiul Botoșani'},
 {'url': 'https://www.gazetabt.ro/singura-ferma-bio-de-gaini-ouatoare-din-regiunea-moldovei-la-botosani-investitie

In [5]:
all_articles = []

i = 1
# allowed_zeros = 10
while True:
    url = url_pages + '/page/' + str(i)
    
    new_arts = get_all_articles(url, headers)
    if len(new_arts) == 0:
        break
    all_articles.extend(new_arts)
    # sleep(0.5)
    
    print(str(len(all_articles)))
    if len(all_articles) >= num_arts:
        break
    
    i += 1
    

30
60
90
120
150
180
210
240
270
300


In [6]:
with open('urls.txt', 'w') as f:
    json.dump(all_articles, f, indent=4)

In [7]:
def get_article_contents(url: str, headers: dict) -> str:
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    return soup.prettify()

In [8]:
for i in tqdm(range(len(all_articles))):
    all_articles[i]['content'] = get_article_contents(all_articles[i]['url'], headers)

100%|██████████| 300/300 [10:29<00:00,  2.10s/it]


In [9]:
for i in tqdm(range(len(all_articles))):
    with open(f'articles/{newspaper_name}_{"%04d" % i}.html', 'w') as f:
        f.write(all_articles[i]['content'])
    with open(f'titles/{newspaper_name}_{"%04d"% i}.txt', 'w') as f:
        f.write(all_articles[i]['title'])
    with open(f'urls/{newspaper_name}_{"%04d"% i}.txt', 'w') as f:
        f.write(all_articles[i]['url'])

100%|██████████| 300/300 [00:00<00:00, 1102.31it/s]
